In [1]:
import os 
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

d:\Agnetic AI\9am\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
groq_url = "https://api.groq.com/openai/v1"
groq = OpenAI(
    base_url=groq_url,
    api_key= groq_api_key
)

In [3]:
model = "openai/gpt-oss-120b"
client = groq

In [10]:
system_prompt = """
You are a senior backend engineer.

Given this error message, do the following:
1. Generate a short Google search query
2. Detect the tech stack if possible
3. List possible causes
4. Suggest fixes

Ignore any questions not related to backend engineering or frontend development.
"""

In [5]:
def stream_ai_response(error_message):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": error_message}
    ]

    stream = client.chat.completions.create(
        model= model, messages=messages, stream=True
    )

    response = ""
    for chunk in stream:
        if not getattr(chunk, "choices", None):
            continue

        choice = chunk.choices[0]
        delta = getattr(choice, "delta", None)
        if not delta:
            continue

        text = getattr(delta, "content", None)
        if not text:
            continue

        response += text
        yield response

In [8]:
error_input = gr.Textbox(label="Error message", lines=10)
output = gr.Markdown(label="AI Response")
view = gr.Interface(
    fn = stream_ai_response,
    inputs = [error_input],
    outputs = [output],
    flagging_mode="never"
)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
